In [34]:
from osgeo import gdal, ogr, osr
import geopandas as gpd
import glob
import os
import numpy as np
import math
import pandas as pd
import rasterio.mask



In [9]:
fieldPolygons = gpd.read_file("./spatial_data/field_polygons.geojson")
fieldPolygons["Name"] = fieldPolygons.apply(lambda row: f"{row['Name']}_{row.name}", axis=1)
print(fieldPolygons.head())


   id         Name                                           geometry
0   1       Pin4_0  MULTIPOLYGON (((30.17411 49.99641, 30.17705 50...
1   2      Pin17_1  MULTIPOLYGON (((30.24050 49.98769, 30.24404 49...
2   3  Shev7-8-9_2  MULTIPOLYGON (((30.32204 50.00902, 30.33148 50...
3   4     Shev10_3  MULTIPOLYGON (((30.35295 50.01389, 30.35007 50...
4   5        Tr5_4  MULTIPOLYGON (((30.33304 50.02297, 30.34076 50...


In [8]:
directory = "./spatial_data/"
tifFiles = glob.glob(directory + "**/*.tif")
numTifFiles = len(tifFiles)
print("Number of .tif files:", numTifFiles)



Number of .tif files: 68


In [11]:
fieldPolygons = gpd.read_file("./spatial_data/field_polygons.geojson")
resampledDirectory = "./resampled/"


for idx, row in fieldPolygons.iterrows():
    directoryName = os.path.join(resampledDirectory, row["Name"])
    os.makedirs(directoryName, exist_ok=True)

In [25]:
mainFolder = "./spatial_data/"
fieldPolygonsFile = "./spatial_data/field_polygons.geojson"
resampledDirectory = "./resampled/"
fieldPolygons = gpd.read_file(fieldPolygonsFile)

for folder in os.listdir(mainFolder):
   
    folderPath = os.path.join(mainFolder, folder)

   
    for file_name in os.listdir(folderPath):
       
        if file_name.endswith(".tif"):
            
            inputRaster = os.path.join(folderPath, file_name)

        
            ds = gdal.Open(inputRaster)

           
            for idx, row in fieldPolygons.iterrows():
                
                fieldName = row["Name"]
                fieldDirectory = os.path.join(resampledDirectory, folder, fieldName)

          
                os.makedirs(fieldDirectory, exist_ok=True)

                fieldGeom = row.geometry

                transform = ds.GetGeoTransform()
                pixelSize = transform[1]

                
                x_offset = int(math.floor((fieldGeom.bounds[0] - transform[0]) / pixelSize))
                y_offset = int(math.floor((transform[3] - fieldGeom.bounds[3]) / pixelSize))
                x_size = int(math.floor((fieldGeom.bounds[2] - fieldGeom.bounds[0]) / pixelSize))
                y_size = int(math.floor((fieldGeom.bounds[3] - fieldGeom.bounds[1]) / pixelSize))

                
                band = ds.GetRasterBand(1)
                data = band.ReadAsArray(x_offset, y_offset, x_size, y_size)

                
                driver = gdal.GetDriverByName("GTiff")
                outputRaster = os.path.join(fieldDirectory, f"{file_name.split('.')[0]}_{fieldName}.tif")
                outDs = driver.Create(outputRaster, x_size, y_size, 1, gdal.GDT_Float32)

                
                outDs.SetProjection(ds.GetProjection())
                outDs.SetGeoTransform(transform)

                
                outBand = outDs.GetRasterBand(1)
                outBand.WriteArray(data)

               
                outDs = None

            
            ds = None




AttributeError: 'NoneType' object has no attribute 'SetProjection'

Код який зверху працює, але не так як хотілось. Він бере першу папку з тіфом, перезаписує її в resampled і створює там папки з обрізаними tif. Як це виглядає є в скріншоті до даної лабораторної :(